[Nuoroda į google colab platformą](https://colab.research.google.com/drive/1iTcxVRN_MJ3gbq3oEXy2sQIQtGfU3I8i?usp=sharing)

In [ ]:
!pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o==3.38.0.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html
     |████████████████████████████████| 177.2 MB 26 kB/s 
  Created wheel for h2o: filename=h2o-3.38.0.1-py2.py3-none-any.whl size=177276478 sha256=9a107b1696a3cd17ba461fc753f197ca39420e02388679626edfb074e5f07703
  Stored in directory: /root/.cache/pip/wheels/51/91/49/126d827f001a62a462df56cb6d9acaf7ff8c3c0b0e2e48ef13
Successfully built h2o


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import h2o
from h2o.grid.grid_search import H2OGridSearch
from h2o.estimators.gbm import H2OGradientBoostingEstimator
RANDOM_STATE = 8

### Preparing for training

In [ ]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.17" 2022-10-18; OpenJDK Runtime Environment (build 11.0.17+8-post-Ubuntu-1ubuntu218.04); OpenJDK 64-Bit Server VM (build 11.0.17+8-post-Ubuntu-1ubuntu218.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.7/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpzy_m01jn
  JVM stdout: /tmp/tmpzy_m01jn/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpzy_m01jn/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,05 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.2
H2O_cluster_version_age:,26 days
H2O_cluster_name:,H2O_from_python_unknownUser_f88ys7
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.172 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [ ]:
# h2o.H2OFrame(pdf_merged)
hdf = h2o.import_file("/content/drive/MyDrive/KTU/KTU_Challenge/sample_data_additional_features.csv")

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [ ]:
y = "y"
x = [
    #'id',
 'amount_current_loan',
 'term',
 'credit_score',
 'loan_purpose',
 'yearly_income',
 'home_ownership',
 'bankruptcies',
 'years_current_job',
 'monthly_debt',
 'years_credit_history',
 'months_since_last_delinquent',
 'open_accounts',
 'credit_problems',
 'credit_balance',
 'max_open_credit']

In [ ]:
# Split data into train & validation
ss = hdf.split_frame(ratios=[.7, .2], seed = 8)
train_ss = ss[0]
train_ss["y"] = train_ss["y"].asfactor()
valid_ss = ss[1]
valid_ss["y"] = valid_ss["y"].asfactor()
test_ss = ss[2]
test_ss["y"] = test_ss["y"].asfactor()

# Geriausias GBM modelis

In [ ]:
gbm_model = H2OGradientBoostingEstimator(ntrees=100,
                                         learn_rate=0.1,
                              max_depth=32,
                              sample_rate=0.8,
                              col_sample_rate=0.8,
                              stopping_tolerance=0.01,
                              seed=8
                                          )

hyper_params = {
    'learn_rate': [0.0, 0.1, 0.25,0.5,1.0],
    "sample_rate": [0.8, 1.0],
    "col_sample_rate": [0.8, 1.0],
    'max_depth': [28, 30, 32]
                }
# geriausi alpha 0.0 ir 0.1
search_criteria = {'strategy' : "RandomDiscrete", 
                        'max_models' : 100,
                        'max_runtime_secs' : 900,
                        'stopping_tolerance' : 0.001,
                        'stopping_rounds' : 15,
                        'seed' : 8}

In [ ]:
gbm_grid = H2OGridSearch(gbm_model, hyper_params=hyper_params, search_criteria = search_criteria)
%time gbm_grid.train(x = x, y = y, training_frame = train_ss, validation_frame = valid_ss)

gbm Grid Build progress: |██████

In [ ]:
gbm_model.train(x = x, y = y, training_frame = train_ss, validation_frame = valid_ss)

gbm Model Build progress: |

/usr/local/lib/python3.8/dist-packages/h2o/estimators/estimator_base.py:193: RuntimeWarning: Stopping tolerance is ignored for _stopping_rounds=0.
  warnings.warn(mesg["message"], RuntimeWarning)


██████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_model_python_1670871642062_1


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    100                100                         1.2989e+08             1            32           28.97         2             242631        103478

ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.03097534809859576
RMSE: 0.17599814799763025
LogLoss: 0.13676641081887514
Mean Per-Class Error: 0.025244292737255657
AUC: 0.9984223763317601
AUCPR: 0.9953016630134275
Gini: 0.9968447526635202

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.37382189602939303
       0            1            Error    Rate
-----  -----------  -----------  -------  --------------------
0      5.24759e+06  57411        0.0108   (57411.0/5305005.0)
1      67238        1.62784e+06  0.0397   (67238.0/1695081.0)
Total  5.31483e+06  1.68525e+06  0.0178   (124649.0/7000086.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value        idx
---------------------------  -----------  -----------  -----
max f1                       0.373822     0.963125     206
max f2                       0.29184      0.972969     229
max f0point5                 0.459396     0.973309     183
max accuracy                 0.373822     0.982193     206
max precision                0.99879      1            0
max recall                   0.0630432    1            346
max specificity              0.99879      1            0
max absolute_mcc             0.373822     0.951395     206
max min_per_class_accuracy   0.318079     0.978982     221
max mean_per_class_accuracy  0.29184      0.979696     229
max tns                      0.99879      5.30500e+06  0
max fns                      0.99879      1.57228e+06  0
max fps                      0.000796006  5.30500e+06  399
max tps                      0.0630432    1.69508e+06  346
max tnr                      0.99879      1            0
max fnr                      0.99879      0.927556     0
max fpr                      0.000796006  1            399
max tpr                      0.0630432    1            346

Gains/Lift Table: Avg response rate: 24.22 %, avg score: 24.21 %
group    cumulative_data_fraction    lower_threshold    lift         cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  -----------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0100242                   0.998771           4.12965      4.12965            1                0.998978     1                           0.998978            0.0413963       0.0413963                  312.965   312.965            0.0413963
2        0.0200215                   0.998232           4.12965      4.12965            1                0.99852      1                           0.998749            0.0412853       0.0826816                  312.965   312.965            0.0826816
3        0.0300002                   0.997955           4.12965      4.12965            1                0.998118     1                           0.998539            0.0412087       0.12389                    312.965   312.965            0.12389
4        0.0403382                   0.995821           4.12965      4.12965            1                0.997151     1                           0.998183            0.0426924       0.166583                  

In [ ]:
gbm_model.model_performance(valid_ss)

ModelMetricsBinomial: gbm
** Reported on test data. **

MSE: 0.1077187464353231
RMSE: 0.32820534187505707
LogLoss: 0.34668344573613724
Mean Per-Class Error: 0.21229305907338636
AUC: 0.8742780496686918
AUCPR: 0.7627092559518026
Gini: 0.7485560993373837

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.29172331892997094
       0            1       Error    Rate
-----  -----------  ------  -------  --------------------
0      1.34358e+06  171960  0.1135   (171960.0/1515542.0)
1      150624       333508  0.3111   (150624.0/484132.0)
Total  1.49421e+06  505468  0.1613   (322584.0/1999674.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value        idx
---------------------------  -----------  -----------  -----
max f1                       0.291723     0.674026     220
max f2                       0.140802     0.731579     297
max f0point5                 0.51858      0.727858     139
max accuracy                 0.441033     0.85453      165
max precision                0.998791     1            0
max recall                   0.000794822  1            399
max specificity              0.998791     1            0
max absolute_mcc             0.398966     0.57628      179
max min_per_class_accuracy   0.186431     0.789291     270
max mean_per_class_accuracy  0.221835     0.792876     252
max tns                      0.998791     1.51554e+06  0
max fns                      0.998791     450506       0
max fps                      0.000794822  1.51554e+06  399
max tps                      0.000794822  484132       399
max tnr                      0.998791     1            0
max fnr                      0.998791     0.930544     0
max fpr                      0.000794822  1            399
max tpr                      0.000794822  1            399

Gains/Lift Table: Avg response rate: 24.21 %, avg score: 22.38 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0100331                   0.998749           4.13043   4.13043            1                0.998972     1                           0.998972            0.0414412       0.0414412                  313.043   313.043            0.0414412
2        0.0200063                   0.998184           4.13043   4.13043            1                0.998502     1                           0.998738            0.0411933       0.0826345                  313.043   313.043            0.0826345
3        0.0300049                   0.997861           4.13043   4.13043            1                0.998091     1                           0.998522            0.0412987       0.123933                   313.043   313.043            0.123933
4        0.040026                    0.995821           4.13043   4.13043            1                0.997089     1                           0.998163            0.0413916       0.165325                   313.043   313.043            0.165325
5        0.0500002                   0.932778           4.0623    4.11684            0.983505         0.97385      0.996709                    0.993313            0.0405179       0.205843                   306.23    311.684            0.205626
6        0.1                         0.689668           3.34123   3.72903            0.808929         0.799959     0.902819                    0.896636            0.167062        0.372904                   234.123   272.903            0.360082
7        0.15                        0.508687           2.66532   3.37447            0.64529          0.595029     0.816977 

## Testing the model

In [ ]:
model = h2o.load_model("/content/drive/MyDrive/KTU/KTU_Challenge/Karolinos_GBM_20221211")

In [ ]:
model.predict(test_ss)

gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


predict,p0,p1
0,0.926625,0.0733754
0,0.950415,0.0495846
0,0.961266,0.0387341
0,0.773264,0.226736
0,0.975296,0.0247044
0,0.937825,0.0621753
0,0.977892,0.0221082
0,0.87136,0.12864
0,0.92602,0.0739801
0,0.981775,0.0182249


In [ ]:
model.model_performance(test_ss)

ModelMetricsBinomial: gbm
** Reported on test data. **

MSE: 0.10709941783729185
RMSE: 0.3272604739917301
LogLoss: 0.3537884848623537
Mean Per-Class Error: 0.20549684441097044
AUC: 0.8867924978101134
AUCPR: 0.779008256465316
Gini: 0.7735849956202268

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.21330049490773523
       0       1       Error    Rate
-----  ------  ------  -------  --------------------
0      689880  68166   0.0899   (68166.0/758046.0)
1      77761   164432  0.3211   (77761.0/242193.0)
Total  767641  232598  0.1459   (145927.0/1000239.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.2133       0.69265   250
max f2                       0.0870494    0.745609  321
max f0point5                 0.45311      0.760951  168
max accuracy                 0.339336     0.865398  203
max precision                0.987753     0.992754  0
max recall                   0.000945693  1         399
max specificity              0.987753     0.999997  0
max absolute_mcc             0.321418     0.607681  209
max min_per_class_accuracy   0.113903     0.801171  302
max mean_per_class_accuracy  0.136912     0.803924  289
max tns                      0.987753     758044    0
max fns                      0.987753     241919    0
max fps                      0.000945693  758046    399
max tps                      0.000945693  242193    399
max tnr                      0.987753     0.999997  0
max fnr                      0.987753     0.998869  0
max fpr                      0.000945693  1         399
max tpr                      0.000945693  1         399

Gains/Lift Table: Avg response rate: 24.21 %, avg score: 17.89 %
group    cumulative_data_fraction    lower_threshold    lift         cumulative_lift    response_rate    score       cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  -----------  -----------------  ---------------  ----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0100006                   0.940619           3.98377      3.98377            0.964611         0.959989    0.964611                    0.959989            0.0398401       0.0398401                  298.377   298.377            0.0393731
2        0.0200002                   0.912724           3.99036      3.98707            0.966207         0.926045    0.965409                    0.943018            0.0399021       0.0797422                  299.036   298.707            0.0788293
3        0.0300008                   0.889318           3.99038      3.98817            0.96621          0.90093     0.965676                    0.928988            0.0399062       0.119648                   299.038   298.817            0.11829
4        0.0400004                   0.866122           3.97963      3.98603            0.963607         0.877756    0.965159                    0.916181            0.0397947       0.159443                   297.963   298.603            0.157604
5        0.05                        0.842317           3.93957      3.97674            0.953909         0.854373    0.962909                    0.90382             0.0393942       0.198837                   293.957   297.674            0.19639
6        0.1                         0.67121            3.7347       3.85572            0.904303         0.766551    0.933606                    0.835186            0.186735        0.385573                   273.47    285.572            0.376812
7        0.15                        0.428054           2.86482      3.52542            0.693674         0.547691    0.853628                    0.739354            0.143241        0.528814           

# Predictions on test data

In [ ]:
test_data = h2o.import_file("/content/drive/MyDrive/KTU/KTU_Challenge/test_data.csv")

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [ ]:
test_data

id,amount_current_loan,term,credit_score,loan_purpose,yearly_income,home_ownership,bankruptcies,years_current_job,monthly_debt,years_credit_history,months_since_last_delinquent,open_accounts,credit_problems,credit_balance,max_open_credit
1,438812,short,fair,debt_consolidation,1.42101e+06,mortgage,0,10,28596,21,nan,11,0,599545,826034
2,278344,long,NA,debt_consolidation,723216,mortgage,0,0,15784.6,16,nan,9,0,271225,449504
3,155716,short,very_good,home_improvements,1.95632e+06,mortgage,0,8,21586.8,27,nan,12,0,28709,314468
4,442992,long,good,debt_consolidation,1.33918e+06,mortgage,0,10,23030.3,17,nan,7,0,766878,982234
5,412346,long,NA,debt_consolidation,nan,mortgage,0,6,24058.9,10,68,11,0,524001,620950
6,225126,short,good,debt_consolidation,1.26378e+06,mortgage,0,4,33339.1,23,63,9,0,122094,257818
7,1e+08,short,good,other,991192,mortgage,0,3,3524.69,16,14,6,0,16112,259160
8,551100,long,good,debt_consolidation,2.87884e+06,rent,1,10,27156.1,15,nan,12,1,303202,405504
9,1e+08,short,very_good,debt_consolidation,1.1579e+06,mortgage,0,0,8085.26,19,nan,6,0,95437,120692
10,133012,short,very_good,debt_consolidation,957315,rent,1,10,17055.2,27,3,9,1,28158,204446


In [ ]:
predictions_gbm = gbm_model.predict(test_data)

gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


/usr/local/lib/python3.8/dist-packages/h2o/job.py:83: UserWarning: Test/Validation dataset column 'credit_score' has levels not trained on: ["NA"]
  warnings.warn(w)


In [ ]:
predictions_df = h2o.as_list(predictions_gbm)

In [ ]:
predictions_df[['p0']].rename(columns={'p0':'y'}).to_csv('Karolina_Laurynas_20221212.csv', index_label='id')

In [ ]:
predictions_df

,predict,p0,p1
0,0,0.904353,0.095647
1,1,0.002140,0.997860
2,0,0.972019,0.027981
3,1,0.681775,0.318225
4,1,0.202093,0.797907
...,...,...,...
9995,0,0.955524,0.044476
9996,0,0.999190,0.000810
9997,0,0.738781,0.261219
9998,0,0.796248,0.203752


In [ ]:
h2o.save_model(gbm_model, 'Karolina_Laurynas_20221212')

'/content/Karolina_Laurynas_20221212/GBM_model_python_1670871642062_1'